In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key= os.getenv("AUTH")
)
        

In [58]:
model1="deepseek/deepseek-chat-v3.1:free"
model2="qwen/qwen3-235b-a22b:free"
model3="openai/gpt-oss-20b:free"

In [25]:
import pandas as pd
kk = pd.read_excel("kk.xlsx")

def parse_range(value):
    """Преобразует строку с диапазоном в среднее число"""
    if pd.isna(value):
        return None
    try:
        # Заменяем запятые на точки и разделяем по дефису
        parts = str(value).replace(',', '.').split('-')
        # Убираем пробелы и преобразуем в числа
        numbers = [float(part.strip()) for part in parts]
        # Возвращаем среднее значение
        return sum(numbers) / len(numbers)
    except:
        return None

# Применяем ко всем столбцам с диапазонами
kk['squirrels_avg'] = kk['squirrels'].apply(parse_range)
kk['fats_avg'] = kk['fats'].apply(parse_range)
kk['carbohydrates_avg'] = kk['carbohydrates'].apply(parse_range)
kk['callories_avg'] = kk['callories'].apply(parse_range)
kk

,Product,squirrels,fats,carbohydrates,callories,squirrels_avg,fats_avg,carbohydrates_avg,callories_avg
0,Хлеб пшеничный из муки высшего сорта,"7,5-7,6","0,8-2,9","49,2-51,4",235-262,7.55,1.85,50.3,248.5
1,Хлеб пшеничный из муки 1 сорта,7.6,0.9,49.7,226,7.60,0.90,49.7,226.0
2,Хлеб из ржано-пшеничной муки,6.8,1.2,46.4,215,6.80,1.20,46.4,215.0
3,Хлеб ржаной,4.7,0.7,49.8,214,4.70,0.70,49.8,214.0
4,"Хлеб ""Бородинский""",6.9,1.3,40.9,208,6.90,1.30,40.9,208.0
...,...,...,...,...,...,...,...,...,...
631,Саке,0.5,-,5,134,0.50,NaN,5.0,134.0
632,Самогон,0.1,0.1,0.4,235,0.10,0.10,0.4,235.0
633,Спирт этиловый 96%,4,-,3.8,710,4.00,NaN,3.8,710.0
634,Текила,1.4,0.3,24,231,1.40,0.30,24.0,231.0


In [26]:
from pydantic import BaseModel, Field
from typing import Optional

class ProductSearch(BaseModel):
    """Эта функция позволяет искать продукты по одному или нескольким параметрам."""

    name: Optional[str] = Field(description="Название продукта", default=None)
    squirrels: Optional[str] = Field(description="Белки", default=None)
    fats: Optional[str] = Field(description="Жиры", default=None)
    carbohydrates: Optional[str] = Field(description="Углеводы", default=None)
    callories: Optional[str] = Field(description="Энергия", default=None)
    squirrels_avg: Optional[str] = Field(description="Белки среднее значение", default=None)
    fats_avg: Optional[str] = Field(description="Жиры среднее значение", default=None)
    carbohydrates_avg: Optional[str] = Field(description="Углеводы среднее значение", default=None)
    callories_avg: Optional[str] = Field(description="Энергия среднее значение", default=None)
    
    # Добавляем параметры для диапазонного поиска
    min_squirrels: Optional[float] = Field(description="Минимальное количество белков", default=None)
    max_squirrels: Optional[float] = Field(description="Максимальное количество белков", default=None)
    min_fats: Optional[float] = Field(description="Минимальное количество жиров", default=None)
    max_fats: Optional[float] = Field(description="Максимальное количество жиров", default=None)
    min_carbohydrates: Optional[float] = Field(description="Минимальное количество углеводов", default=None)
    max_carbohydrates: Optional[float] = Field(description="Максимальное количество углеводов", default=None)
    min_callories: Optional[float] = Field(description="Минимальная энергетическая ценность", default=None)
    max_callories: Optional[float] = Field(description="Максимальная энергетическая ценность", default=None)
    
    sort_order: Optional[str] = Field(
        description="Порядок сортировки (high_protein, low_protein, high_fat, low_fat, high_carbs, low_carbs)",
        default=None,
    )

In [ ]:
tools1 = [
    {
        "type": "function",
        "function": {
            "name": "search_products_by_nutrition",
            "description": "Вызывай, когда пользовать ищет продукты в базе по белкам, жирам, углеводам или каллориям или по названию или по виду",
            "parameters": ProductSearch.model_json_schema(),
            }
        }
    
]

instruction = """
Ты - опытный диетолог и специалист по питанию, в задачу которого входит помогать пользователям подбирать продукты 
по их пищевой ценности: калориям, белкам, жирам и углеводам (БЖУ).

Используй функцию search_products_by_nutrition когда:
- Пользователь запрашивает конкретные продукты по параметрам БЖУ
- Нужно найти продукты в определенном диапазоне калорий
- Требуется подбор по соотношению белков/жиров/углеводов

Если запрос пользователя недостаточно конкретен - вежливо уточни необходимые параметры.
"""

try:
    res = client.chat.completions.create(
        model=model2,
        tools=tools1,
        messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": "Привет! Что посоветуешь, где есть 20 грамм белка и меньше 10 грамм жиров на 100 грамм продукта. Хотелось бы что-то мясное"}
        ],
        max_tokens=1000
    )
    
    print(res.choices[0].message.content)
    
    # Проверяем, хочет ли модель вызвать функцию
    if hasattr(res.choices[0].message, 'tool_calls') and res.choices[0].message.tool_calls:
        print("\nМодель хочет вызвать функцию с параметрами:")
        for tool_call in res.choices[0].message.tool_calls:
            print(f"Функция: {tool_call.function.name}")
            print(f"Аргументы: {tool_call.function.arguments}")
    
except Exception as e:
    print(f"Ошибка: {e}")

<think>Хорошо, пользователь спрашивает о продуктах, которые содержат 20 грамм белка и меньше 10 грамм жиров на 100 грамм. Ему хочется что-то мясное. Нужно использовать функцию search_products_by_nutrition.

Сначала проверяю параметры: белки должны быть 20 г, жиры меньше 10 г. Указано, что на 100 г продукта. Также важно, чтобы продукт был мясным. 

В параметрах функции squirrels – это белки, fats – жиры. Нужно задать squirrels как "20", fats как "<10". Но как точно передать диапазоны? Возможно, параметры принимают строковые значения, например, "20" для точного значения белка, и "<10" для жиров. 

Пользователь не указал калории или углеводы, поэтому их можно не трогать. Также нужно отсортировать, возможно, по мясным продуктам, но в параметрах sort_order есть варианты. Но, возможно, фильтрация по названию (name) не требуется, так как пользователь просит именно мясное, но функция может искать по БЖУ, а потом в результатах отбирать мясные. Однако в функции есть параметр name, но пользовател

In [27]:
kk.head(1)

,Product,squirrels,fats,carbohydrates,callories,squirrels_avg,fats_avg,carbohydrates_avg,callories_avg
0,Хлеб пшеничный из муки высшего сорта,"7,5-7,6","0,8-2,9","49,2-51,4",235-262,7.55,1.85,50.3,248.5


In [28]:
def find_product(req):
    x = kk.copy()
    
    # Поиск по диапазону белков
    if req.min_squirrels:
        x = x[x["squirrels_avg"] >= req.min_squirrels]
    if req.max_squirrels:
        x = x[x["squirrels_avg"] <= req.max_squirrels]
    
    # Поиск по диапазону жиров
    if req.min_fats:
        x = x[x["fats_avg"] >= req.min_fats]
    if req.max_fats:
        x = x[x["fats_avg"] <= req.max_fats]
    
    # Поиск по диапазону углеводов
    if req.min_carbohydrates:
        x = x[x["carbohydrates_avg"] >= req.min_carbohydrates]
    if req.max_carbohydrates:
        x = x[x["carbohydrates_avg"] <= req.max_carbohydrates]
    
    # Поиск по названию
    if req.name:
        x = x[x["Product"].apply(lambda product_name: req.name.lower() in str(product_name).lower())]
    
    # Сортировка
    if req.sort_order and len(x) > 0:
        if req.sort_order == "low_protein":
            x = x.sort_values(by="squirrels_avg")
        elif req.sort_order == "high_protein":
            x = x.sort_values(by="squirrels_avg", ascending=False)
        elif req.sort_order == "low_fat":
            x = x.sort_values(by="fats_avg")
        elif req.sort_order == "high_fat":
            x = x.sort_values(by="fats_avg", ascending=False)
        elif req.sort_order == "low_carbs":
            x = x.sort_values(by="carbohydrates_avg")
        elif req.sort_order == "high_carbs":
            x = x.sort_values(by="carbohydrates_avg", ascending=False)
    
    if len(x) == 0:
        return "Подходящих продуктов не найдено"
    
    return "Вот какие продукты были найдены:\n" + "\n".join(
        [
            f"{z['Product']} - Белки: {z.get('squirrels', 'N/A')}г, Жиры: {z.get('fats', 'N/A')}г, Углеводы: {z.get('carbohydrates', 'N/A')}г"
            for _, z in x.head(10).iterrows()
        ]
    )

# Примеры с диапазонами:
print(find_product(ProductSearch(min_squirrels="20", max_fats="10")))  # Белка ≥20г, жиров ≤10г


Вот какие продукты были найдены:
Горох зерно - Белки: 20.5г, Жиры: 2г, Углеводы: 49.5г
Горох целый шлифованный - Белки: 22г, Жиры: 2г, Углеводы: 57г
Горох колотый - Белки: 23г, Жиры: 1.6г, Углеводы: 48.1г
Маш - Белки: 23.9г, Жиры: 1.2г, Углеводы: 62.6г
Нут - Белки: 20.5г, Жиры: 4.3г, Углеводы: 63г
Фасоль обыкновенная красная, все виды - Белки: 23.5г, Жиры: 0.8г, Углеводы: 60г
Чечевица - Белки: 24.6г, Жиры: 1.1г, Углеводы: 63.4г
Чечевица красная - Белки: 21.6г, Жиры: 1.1г, Углеводы: 48г
Чечевица красная колотая персидская ("Мистраль") - Белки: 24.7г, Жиры: 1.2г, Углеводы: 62.5г
Творог нежирный - Белки: 22г, Жиры: 0.6г, Углеводы: 3.3г


In [29]:
class ProductSearch(BaseModel):
    """Эта функция позволяет искать продукты по одному или нескольким параметрам."""

    name: Optional[str] = Field(description="Название продукта", default=None)
    squirrels: Optional[str] = Field(description="Белки", default=None)
    fats: Optional[str] = Field(description="Жиры", default=None)
    carbohydrates: Optional[str] = Field(description="Углеводы", default=None)
    callories: Optional[str] = Field(description="Энергия", default=None)
    squirrels_avg: Optional[str] = Field(description="Белки среднее значение", default=None)
    fats_avg: Optional[str] = Field(description="Жиры среднее значение", default=None)
    carbohydrates_avg: Optional[str] = Field(description="Углеводы среднее значение", default=None)
    callories_avg: Optional[str] = Field(description="Энергия среднее значение", default=None)
    
    # Добавляем параметры для диапазонного поиска
    min_squirrels: Optional[float] = Field(description="Минимальное количество белков", default=None)
    max_squirrels: Optional[float] = Field(description="Максимальное количество белков", default=None)
    min_fats: Optional[float] = Field(description="Минимальное количество жиров", default=None)
    max_fats: Optional[float] = Field(description="Максимальное количество жиров", default=None)
    min_carbohydrates: Optional[float] = Field(description="Минимальное количество углеводов", default=None)
    max_carbohydrates: Optional[float] = Field(description="Максимальное количество углеводов", default=None)
    min_callories: Optional[float] = Field(description="Минимальная энергетическая ценность", default=None)
    max_callories: Optional[float] = Field(description="Максимальная энергетическая ценность", default=None)
    
    sort_order: Optional[str] = Field(
        description="Порядок сортировки (high_protein, low_protein, high_fat, low_fat, high_carbs, low_carbs)",
        default=None,
    )
    what_to_return: str = Field(
        description="Что вернуть (продукт или его белки, жиры, углеводы)", default=None
    )

    def process(self, session_id):
        return find_product(self)

In [30]:
handover = False

class Handover(BaseModel):
    """Эта функция позволяет передать диалог человеку-оператору поддержки"""

    reason: str = Field(
        description="Причина для вызова оператора", default="не указана"
    )

    def process(self, session_id):
        global handover
        handover = True
        return f"Я побежала вызывать оператора, ваш {session_id=}, причина: {self.reason}"

In [82]:
carts = {}

class AddToCart(BaseModel):
    """Эта функция позволяет добавить продукты в корзину"""

    product_name: str = Field(
        description="Точное название еды, чтобы положить в корзину", default=None
    )
    count: int = Field(
        description="Количество, которое нужно положить в корзину",
        default=100,
    )
   

    def process(self, session_id):
        if session_id not in carts:
            carts[session_id] = []
        carts[session_id].append(self)
        return f"Продукт {self.wine_name} добавлен в корзину: количество грамм {self.count}"

In [92]:
from pydantic import BaseModel, Field
import json  # Добавляем импорт json
from typing import Optional

class ShowShoppingList(BaseModel):
    """Показать текущий список покупок"""
    
    def process(self, session_id):
        try:
            
            
            if not carts:
                return "Ваша корзина пуста"
            
            result = "Ваш список покупок:\n"
            for item in carts:
                result += f"• {item['product']} - {item['quantity']}\n"
            
            return result
        except Exception as e:
            return f"Ошибка при получении корзины: {str(e)}"


In [93]:
class Agent():
    user_sessions = {}

    def __init__(self, instruction, tools=[], session_id='default', model=model2, tool_choice='auto'):
        self.instruction = instruction
        self.model = model
        self.tool_choice = tool_choice
        self.tool_map = {x.__name__: x for x in tools if hasattr(x, '__base__') and issubclass(x, BaseModel)}
        self.tools = self._prepare_tools(tools)
        
        if session_id not in self.user_sessions:
            self.user_sessions[session_id] = {
                "last_reply_id": None,
                "history": [],
            }

    def _prepare_tools(self, tools):
        """Подготавливает инструменты в правильном формате для OpenAI API"""
        formatted_tools = []
        for tool in tools:
            if hasattr(tool, '__base__') and issubclass(tool, BaseModel):
                # Для Pydantic моделей создаем функцию-инструмент
                formatted_tools.append({
                    "type": "function",
                    "function": {
                        "name": tool.__name__,
                        "description": tool.__doc__ or f"Функция {tool.__name__}",
                        "parameters": tool.model_json_schema(),
                    }
                })
            elif isinstance(tool, dict):
                # Если уже в правильном формате
                formatted_tools.append(tool)
        return formatted_tools

    def __call__(self, message, session_id='default'):
        s = self.user_sessions.get(session_id, {'last_reply_id': None, 'history': []})
        s['history'].append({'role': 'user', 'content': message})
        
        try:
            # Создаем сообщения для API
            messages = [
                {"role": "system", "content": self.instruction},
                {"role": "user", "content": message}
            ]
            
            # Добавляем историю если есть
            if len(s['history']) > 1:
                messages = [{"role": "system", "content": self.instruction}] + s['history'][-5:]  # последние 5 сообщений
            
            res = client.chat.completions.create(
                model=self.model,
                messages=messages,
                tools=self.tools if self.tools else None,
                tool_choice=self.tool_choice,
                max_tokens=1000
            )
            
            message_obj = res.choices[0].message
            content = message_obj.content or ""

            # Обрабатываем вызов инструментов
            if hasattr(message_obj, 'tool_calls') and message_obj.tool_calls:
                tool_calls = message_obj.tool_calls
                print(f"Обнаружены вызовы инструментов: {[call.function.name for call in tool_calls]}")
                
                tool_outputs = []
                for call in tool_calls:
                    try:
                        fn_class = self.tool_map.get(call.function.name)
                        if fn_class:
                            # Парсим аргументы
                            args = json.loads(call.function.arguments)
                            # Создаем объект и вызываем process
                            fn_instance = fn_class(**args)
                            result = fn_instance.process(session_id)
                        else:
                            result = f"Инструмент {call.function.name} не найден"
                    except Exception as e:
                        result = f"Ошибка выполнения {call.function.name}: {str(e)}"
                    
                    tool_outputs.append({
                        "role": "tool",
                        "tool_call_id": call.id,
                        "content": str(result)
                    })
                
                # Получаем финальный ответ с результатами инструментов
                final_messages = messages + [message_obj] + tool_outputs
                
                final_res = client.chat.completions.create(
                    model=self.model,
                    messages=final_messages,
                    max_tokens=1000
                )
                
                content = final_res.choices[0].message.content
                
            # Сохраняем в историю
            s['history'].append({'role': 'assistant', 'content': content})
            self.user_sessions[session_id] = s
            
            # Возвращаем объект с output_text
            class Response:
                def __init__(self, content):
                    self.output_text = content
            return Response(content)
            
        except Exception as e:
            error_msg = f"Ошибка: {str(e)}"
            s['history'].append({'role': 'assistant', 'content': error_msg})
            self.user_sessions[session_id] = s
            
            class Response:
                def __init__(self, content):
                    self.output_text = content
            return Response(error_msg)

    def history(self, session_id='default'):
        return self.user_sessions[session_id]['history']

In [94]:
instruction = """
Ты - опытный диетолог и специалист по питанию, в задачу которого входит отвечать на вопросы пользователя о продуктах питания, их пищевой ценности и помогать составлять сбалансированный рацион.
ВАЖНО: Не показывай процесс размышления, внутренние мысли или reasoning. 
Просто давай прямой, четкий ответ на вопрос пользователя.
Если вопрос касается поиска конкретных продуктов по пищевой ценности (белки, жиры, углеводы, калории), то вызови функцию ProductSearch.
Для передачи управления оператору - вызови функцию Handover.
Для добавления продуктов в список покупок используй AddToShoppingList.
Для просмотра текущего списка покупок: ShowShoppingList.

Все названия продуктов, категорий и единицы измерения (граммы, калории) указывай на русском языке.
При рекомендациях учитывай цели пользователя: похудение, набор мышечной массы, поддержание веса, специальные диетические потребности.
Основывай рекомендации на принципах сбалансированного питания
"""

cook_agent = Agent(
    instruction=instruction,
    tools=[ProductSearch, Handover, AddToCart, ShowShoppingList],
)

In [ ]:
print(cook_agent("Какие мясные продукты богаты белком и содержат мало жира?").output_text)

In [90]:
print(cook_agent("Добавь Индейка  в корзину, 800 грамм").output_text)

Обнаружены вызовы инструментов: ['AddToCart']
Обнаружена ошибка в формате запроса. Исправляю:  
<tool_call>
{"name": "AddToCart", "arguments": {"product_name": "Индейка", "count": 800}}
</tool_call>


In [91]:
print(cook_agent("Покажи корзину").output_text)

Обнаружены вызовы инструментов: ['ShowCart']
<think>Хорошо, возникла ошибка при выполнении функции ShowCart: "name 'json' is not defined". Это, скорее всего, связано с тем, что в коде, отвечающем за обработку функции ShowCart, используется библиотека json, которая не была импортирована. Но как ассистент, я не могу исправлять код сервера. Моя задача — правильно сформировать запрос на выполнение функции и обработать возможные ошибки.

В предыдущих шагах пользователь добавил "Индейка 800 грамм" в корзину через AddToCart. Теперь при вызове ShowCart возникла ошибка. Возможно, система ожидает, что я проигнорирую ошибку или повторю запрос. Но по правилам, если функция не сработала, нужно сообщить пользователю.

Однако в инструкции сказано, что если запрос связан с поиском продуктов по БЖУ, нужно вызвать ProductSearch. Но здесь запрос "Покажи корзину" явно требует ShowShoppingList. Возможно, ошибка в названии функции: в системе может быть ShowShoppingList вместо ShowCart. В исходном описании ф